In [ ]:
#Import required packages
import cv2 as cv
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

import pandas as pd
from Helper_functions import load_dict
from AI_functions import resnet18,CellDataset_supervised,data_generator
import torch.nn as nn
import torch.optim as optim


In [ ]:
### RETINA_1_2
#Debris and Doublets:
OTHER=[922, 6342, 8158, 8703, 5524, 902, 9760, 8737, 55404, 841, 851, 5277, 5829, 7777, 51964, 9823, 54418, 54239, 6156, 8487, 5415, 50438, 670, 49993, 55353, 7878, 50196, 856, 9421, 52299, 54295, 54603, 54704, 50189, 55290, 7164, 705, 7204, 53440, 55283, 7292, 6151]
SINGLES=[52596, 7186, 55477, 6624, 51312, 7344, 54918, 5978, 53696, 807, 8336, 8475, 55349, 5243, 55714, 8329, 51652, 51648, 50895, 9536, 55141, 50406, 5208, 51799, 9845, 50229, 6622, 8199, 7800, 51898, 9733, 52683, 7414, 49884, 53164, 9868, 7998, 53455, 55694, 9598, 52251, 53414, 7549, 50199, 8152, 5265, 49955, 50998, 50929, 52043]


In [ ]:
DATASETS=["BoneMarrow_sample1"]#,"Retina_1_2","Colon","Choroid"]
DATASET="Retina_0_0"
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\\" +DATASET
outpath = basepath + "\\Outputs"
image_dim=64 #Dim of the final images
nuclear_channel="Ch7"
cellmask_channel="Ch1_mask"
df=pd.read_csv(outpath+"\\cell_info1.csv")
cell_names=df["Cell_ID"].to_numpy()
Prediction_Channels=['Ch07']

In [ ]:
# df=pd.read_csv(outpath+"\\cell_info1.csv")
# df_labels=pd.read_csv(outpath+"\\labeled_cells_final.csv")
# df_labels=df_labels.rename(columns={"cell_id":"Cell_ID"})
# df_labels=df_labels[["Cell_ID","label"]]
# df_merged=pd.merge(df,df_labels,left_on="Cell_ID",right_on="Cell_ID",how='left')
# df_merged.to_csv(outpath+"\\cell_info1.csv")

In [ ]:
image_dict=load_dict(outpath,cell_names,image_dim)

In [ ]:
plt.imshow(image_dict[39010]['Ch1'])

In [ ]:
BUGS=[]
for image in image_dict:
    for i in image_dict[image]:
      #  print(i)
        try: 
            if i==None:
                BUGS.append(image)
            
        except:
            pass

print(len(BUGS))

In [ ]:
Channels=['Ch1']  #Channel to be fed to the NN
images_with_index = []

for image_i in image_dict:
    #print(image_dict[image_i].keys())
    if len(image_dict[image_i].keys())>=len(Channels):
        image=cv.merge([image_dict[image_i][i] for i in Channels])
        images_with_index.append((int(image_i),image))
    else:
        print(image_i)
    
images=np.array([image[1] for image in images_with_index])
names=np.array([image[0] for image in images_with_index])
assert sum(names!=df['Cell_ID'].to_numpy()) ==0  #Check that the order has been preserved
DNA_pos=df["DNA_pos"].to_numpy()
Touches_Boundary=df["Touches_boundary"].to_numpy()
labels=df['label'].to_numpy().copy()
idx_nan=np.array([i is not np.nan for i in labels],dtype=int)
labels[labels=='d']='f'
labels[labels=='f']=0
labels[labels=='s']=1
convert={'0':[0,1],'1':[1,0]}

labels1=[]
for i in labels:
    if i is not np.nan:
        labels1.append(convert[str(i)])
    else:
        labels1.append([10,10])  #Doesnt matter

labels1=np.array(labels1,dtype=object)


Thresh=50
idx_to_keep=idx_nan+np.array(Touches_Boundary==0,dtype=int)+np.array(df["Gradient RMS_M01_Ch01"]>Thresh,dtype=int)==3 #keep dnapos, no touch boundarym APC and Other
#Filter
images=images[idx_to_keep]
names=names[idx_to_keep]
labels=labels1[idx_to_keep]
#labels=np.concatenate(labels,axis=0).reshape(len(labels),2)

mini=int(round(abs(np.array(images).min()),0))
images=images+abs(np.array(images).min())
mean=np.array(images).mean()
maxi=np.array(images).max()
std=np.array(images).std()

In [ ]:
plt.imshow(images[names==39691][0])
labels[names==113]

In [ ]:
labels

In [ ]:
labels=labels1[idx_to_keep]

In [ ]:
df[df["Cell_ID"]==10007]

In [ ]:
i=8
plt.imshow(images[i])
print(names[i])
print(labels[i])
df[df["Cell_ID"]==names[i]]

In [ ]:
returned=data_generator(images,labels,names,mini,train_test_split = 0.8,batch_size = 100,sample=False)
train,test,batch_size,mean_loader,std_loader=returned
[train_data,train_data1,train_labels,train_ID]=train
[test_data,test_data1,test_labels,test_ID]=test

In [ ]:
for i in range(3): 
    plt.imshow(images[i])
    plt.colorbar()
    plt.show()

In [ ]:
transform_train = transforms.Compose(
    [transforms.ToPILImage(),transforms.RandomHorizontalFlip(p=0.5),transforms.RandomVerticalFlip(p=0.5),transforms.RandomRotation(degrees=18,fill=mini),
    transforms.ToTensor(),
    transforms.Normalize(mean_loader,std_loader)  
])

transform_validation = transforms.Compose(
    [transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean_loader,std_loader)  
])

In [ ]:
train_ID

In [ ]:
plt.imshow(train_data1[train_ID==39691][0])

In [ ]:
train_data = CellDataset_supervised(train_data1,train_labels,train_ID, transform_train)
test_data = CellDataset_supervised(test_data1,test_labels, test_ID,transform_validation)

# from torch.utils.data.sampler import WeightedRandomSampler
# print(np.max(train_labels))
# def sampler_f(x):
#     return ((x)+abs(np.min(train_labels)))**3+1
# weights=sampler_f(train_labels)
# #weights/=np.sum(weights)
# torch.tensor(weights)
# sampler = WeightedRandomSampler(weights.T[0], len(weights))


#Create DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size,shuffle=True)#False,sampler=sampler)
validation_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
for i in validation_loader:
    plt.imshow(i[0][0][0])
    plt.show()
    print(i[1][0])
    print(i[2][0])
   # break

In [ ]:
#torch.save(train_loader,outpath+"\\"+DATASET+"_train.pth")
#torch.save(validation_loader,outpath+"\\"+DATASET+"_validation.pth")

In [ ]:
plt.imshow(next(iter(train_loader))[0][0][0],vmin=-10,vmax=10)
plt.colorbar()
plt.show()
plt.imshow(next(iter(train_loader))[0][0][0],vmin=-10,vmax=10)
plt.colorbar()

In [ ]:
def train_epoch(NN, device, dataloader, loss_fn, optimizer,noise_factor=0):
    NN.train()
    train_loss = []
    for image_batch,labels_batch,_ in dataloader: 
        image_noisy = image_batch
        image_batch = image_noisy.to(device)
        labels_batch=labels_batch.to(device)
        output = NN(image_batch)
        loss = loss_fn(labels_batch,output)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.detach().cpu().numpy())
    return np.mean(train_loss)

def validation_epoch(NN, device, dataloader, loss_fn):
    NN.eval()
    val_loss=[]
    with torch.no_grad(): 
        for image_batch,labels_batch,ID_batch in dataloader:
            image_batch = image_batch.to(device)
            labels_batch = labels_batch.to(device)
            output = NN(image_batch)
            loss = loss_fn(labels_batch,output,)
            val_loss.append(loss.detach().cpu().numpy())

    return np.mean(val_loss)


In [ ]:
for i in validation_loader:
    plt.imshow(i[0][0][0])
    plt.show()
    print(i[1][0])
    print(i[2][0])
   # break

In [ ]:
num_epochs=20
lr=1e-3
num_classes=2

ConvNet_simple=resnet18(channel_num=len(Channels),num_classes=num_classes)
#ConvNet_simple=Trained_model

device = "cuda" if torch.cuda.is_available() else "cpu"
ConvNet_simple.to(device)
loss_dict={"L1":nn.L1Loss(),"MSE":nn.MSELoss()}

loss_fn = loss_dict["L1"]

#optimizer = optim.Adam(ConvNet_simple.parameters(), lr = lr) 
optimizer = optim.Adam(filter(lambda p: p.requires_grad, ConvNet_simple.parameters()),lr=lr)

diz_loss = {'train_loss':[],'val_loss':[]}

counter=0

for epoch in range(num_epochs):

    train_loss = train_epoch(ConvNet_simple,device,train_loader,loss_fn,optimizer)
    val_loss = validation_epoch(ConvNet_simple,device,validation_loader,loss_fn)


    print('\n EPOCH',epoch+1,' \t train loss',train_loss,' \t val loss',val_loss)
    diz_loss['train_loss'].append(train_loss)
    diz_loss['val_loss'].append(val_loss)


#_ = loss_over_epochs(diz_loss,num_epochs)

In [ ]:

predictions=np.zeros((len(test_data1),num_classes))
names=np.zeros((len(test_data1)))
labels=np.zeros((len(test_data1),num_classes))
i=0
ConvNet_simple.eval()
with torch.no_grad():
    for X, Y,ID in validation_loader:
        X = X.to(device)
        latents=ConvNet_simple(X).detach().cpu().numpy()
        labels[i:i+len(latents)]=Y
        names[i:i+len(latents)]=ID
        predictions[i:i+len(latents)]=latents
        i+=len(latents)


In [ ]:
labels

In [ ]:
predictions

In [ ]:
a='["a","b"]'

In [ ]:
eval(a)